In [1]:
import os
import pathlib
import pandas as pd
from sklearn.metrics import classification_report
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [2]:
!pip install ktrain
import ktrain
from ktrain import text

     |████████████████████████████████| 25.2 MB 3.1 MB/s 
     |████████████████████████████████| 981 kB 23.9 MB/s 
     |████████████████████████████████| 241 kB 45.8 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
     |████████████████████████████████| 468 kB 44.5 MB/s 
  Created wheel for ktrain: filename=ktrain-0.14.7-py3-none-any.whl size=25240972 sha256=1a7f2d427791c6c26d3d6646350e657395d38a956aa2d967ea9d98239e4f19eb
  Stored in directory: /root/.cache/pip/wheels/d5/f8/64/c482e2e11303d04d85af01b9b94ecfbeff8620be8f6e543e5f
  Created wheel for keras-bert: filename=keras_bert-0.81.0-py3-none-any.whl size=37912 sha256=b78afa90df8a323b03538d85839b106a94408360801e0d03f37ca096a8fa5277
  Stored in directory: /root/.cache/pip/wheels/fc/f6/94/9c54242cde921a3cdc7d049bae3f137d21fa28d3b8ccefd8a0
  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993191 sha256=de2b4428562a2bb71d7558435626e155aa00a532a9a1463f03992ed4ef4a42d2
  Stored in directory: /

In [3]:
#SETTINGS 
#check the commented line for info

train_path="../input/sentimentdatasets/dummy.csv" 
tr_path=pathlib.Path(train_path)
if tr_path.exists ():
    train_df=pd.read_csv(tr_path, header=0) 
    print("Dummy path set.")
else: 
    raise SystemExit("Dummy path does not exist.")   
    
    
model_path="../input/models/stackoverflow_model.h5" #insert the path for the model
mo_path=pathlib.Path(model_path)

if mo_path.exists ():
    print("Model path set.")
else: 
    raise SystemExit("Model path does not exist.")  
    

data_path="../input/sentimentdatasets/testGithub.csv" #insert the path of the data for the prediction
da_path=pathlib.Path(data_path)

#set the dataframe 
#parameters for 
#jiradataset - - - data_path, encoding="utf-16", header=None - - - remember to set test_df[2]=test_df[2].astype(str) due to a dataset bug
#stackoverflowdataset - - - data_path, encoding='utf-16',sep=';', header=None
#githubdataset - - - data_path, sep=';', header=0
if da_path.exists ():
    test_df=pd.read_csv( data_path, sep=';', header=0) 
    data_text=test_df["Text"] #set 2 for StackOverFlow and Jira and "Text" for Github
    data_label=test_df["Polarity"] #set 1 for StackOverFlow and Jira and "Polarity" for Github
    print("Data path set.")
else: 
    raise SystemExit("Data path does not exist.")    
    
  
 


Dummy path set.
Model path set.
Data path set.


In [4]:
train_df.head()

,ID,Text,Polarity
0,1,Ex1,positive
1,2,Ex2,negative
2,3,Ex3,neutral


In [5]:
test_df.head()

,ID,Polarity,Text
0,284853,positive,Thanks! Fixed. You think after all those code ...
1,45017,negative,"Aw, such a shame..."""
2,404962,positive,"Sorry for my last comment vcrx6, thanks for yo..."
3,4185263,neutral,Yes. eg. above `pam['gecos_map'']` is passed t...
4,21077,negative,"sorry, but I don't understand you :/"""


In [6]:
#parameters for 
#jiradataset - - - train_df["Text"], train_df["Polarity"], x_test=test_df[2], y_test=test_df[1], maxlen=500, preprocess_mode='bert', lang="en"
#stackoverflowdataset - - - train_df["Text"], train_df["Polarity"], x_test=test_df[2], y_test=test_df[1], maxlen=500, preprocess_mode='bert', lang="en"
#githubdataset - - - train_df["Text"], train_df["Polarity"], x_test=test_df["Text"], y_test=test_df["Polarity"], maxlen=500, preprocess_mode='bert', lang="en"
(x_train, y_train),(x_test, y_test), preproc=text.texts_from_array(train_df["Text"], train_df["Polarity"], x_test=test_df["Text"], y_test=test_df["Polarity"], maxlen=500, preprocess_mode='bert', lang="en")

task: text regression (supply class_names argument if this is supposed to be classification task)
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [7]:
learner = ktrain.get_learner(text.text_classifier('bert', (x_train, y_train) , preproc=preproc), 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


In [8]:
learner.load_model(model_path)
print("model loaded successfully")

model loaded successfully


In [9]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [10]:
#get the texts and the labels from the test dataset
data=data_text.tolist()
label=data_label.tolist()


In [11]:
#generating a csv result file containing the wrong predictions
i=0
correct=0
wrong=0
total=len(data)
true_lab=[]
pred_lab=[]
text=[]
for dt in data:
    result=predictor.predict(dt)
    
    if not result== label[i]:
        text.append(dt)
        pred_lab.append(result)
        true_lab.append(label[i])
        wrong+=1
    else:
        correct+=1
    
    i+=1

name_dict = {
            'Name': text,
            'Gold Label' : true_lab,
            'Predicted Label': pred_lab
          }

wrong_data= pd.DataFrame(name_dict)

wrong_data.to_csv("wrong_results.csv", sep=';')   
        

        
    
    

In [12]:
names = ['negative', 'neutral', 'positive']
y_pred = predictor.predict(data)
y_true= label
print(classification_report(y_true, y_pred, target_names=names))

              precision    recall  f1-score   support

    negative       0.87      0.70      0.78       626
     neutral       0.74      0.92      0.82       907
    positive       0.90      0.74      0.81       604

    accuracy                           0.81      2137
   macro avg       0.84      0.79      0.80      2137
weighted avg       0.82      0.81      0.80      2137



In [13]:
print("Correct: ", correct,"/",total,"\nWrong: ", wrong,"/",total)

Correct:  1723 / 2137 
Wrong:  414 / 2137
